In [86]:
# Import the modules
import numpy as np
import pandas as pd
from pathlib import Path
from sklearn.metrics import balanced_accuracy_score, confusion_matrix, classification_report
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from imblearn.over_sampling import RandomOverSampler



---

## Split the Data into Training and Testing Sets

### Step 1: Read the `lending_data.csv` data from the `Resources` folder into a Pandas DataFrame.

In [87]:
# Read the CSV file from the Resources folder into a Pandas DataFrame
df_lending_data = pd.read_csv("lending_data.csv")
# Review the DataFrame
df_lending_data.head(10)

,loan_size,interest_rate,borrower_income,debt_to_income,num_of_accounts,derogatory_marks,total_debt,loan_status
0,10700.0,7.672,52800,0.431818,5,1,22800,0
1,8400.0,6.692,43600,0.311927,3,0,13600,0
2,9000.0,6.963,46100,0.349241,3,0,16100,0
3,10700.0,7.664,52700,0.430740,5,1,22700,0
4,10800.0,7.698,53000,0.433962,5,1,23000,0
5,10100.0,7.438,50600,0.407115,4,1,20600,0
6,10300.0,7.490,51100,0.412916,4,1,21100,0
7,8800.0,6.857,45100,0.334812,3,0,15100,0
8,9300.0,7.096,47400,0.367089,3,0,17400,0
9,9700.0,7.248,48800,0.385246,4,0,18800,0


### Step 2: Create the labels set (`y`)  from the “loan_status” column, and then create the features (`X`) DataFrame from the remaining columns.

In [88]:
# Separate the data into labels and features
df_lending_data.dtypes

loan_size           float64
interest_rate       float64
borrower_income       int64
debt_to_income      float64
num_of_accounts       int64
derogatory_marks      int64
total_debt            int64
loan_status           int64
dtype: object

In [89]:
# Separate the y variable, the labels
y = df_lending_data["loan_status"]
# Separate the X variable, the features
X = df_lending_data.iloc[:,:-1]

In [90]:
# Review the y variable Series
y.unique

<bound method Series.unique of 0        0
1        0
2        0
3        0
4        0
        ..
77531    1
77532    1
77533    1
77534    1
77535    1
Name: loan_status, Length: 77536, dtype: int64>

In [91]:
# Review the X variable DataFrame
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 77536 entries, 0 to 77535
Data columns (total 7 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   loan_size         77536 non-null  float64
 1   interest_rate     77536 non-null  float64
 2   borrower_income   77536 non-null  int64  
 3   debt_to_income    77536 non-null  float64
 4   num_of_accounts   77536 non-null  int64  
 5   derogatory_marks  77536 non-null  int64  
 6   total_debt        77536 non-null  int64  
dtypes: float64(3), int64(4)
memory usage: 4.1 MB


### Step 3: Check the balance of the labels variable (`y`) by using the `value_counts` function.

In [92]:
# Check the balance of our target values
y.value_counts()

loan_status
0    75036
1     2500
Name: count, dtype: int64

### Step 4: Split the data into training and testing datasets by using `train_test_split`.

In [93]:
# Split the data using train_test_split
# Assign a random_state of 1 to the function
X_train, X_test, y_train, y_test = train_test_split(X,y, random_state=1)


---

## Create a Logistic Regression Model with the Original Data

###  Step 1: Fit a logistic regression model by using the training data (`X_train` and `y_train`).

In [94]:
# Instantiate the Logistic Regression model
# Assign a random_state parameter of 1 to the model
model_logisticR = LogisticRegression(random_state=1)
# Fit the model using training data
model_logisticR.fit(X_train, y_train)

LogisticRegression(random_state=1)

### Step 2: Save the predictions on the testing data labels by using the testing feature data (`X_test`) and the fitted model.

In [95]:
# Make a prediction using the testing data
y_pred_logisticR = model_logisticR.predict(X_test)

### Step 3: Evaluate the model’s performance by doing the following:

* Calculate the accuracy score of the model.

* Generate a confusion matrix.

* Print the classification report.

In [96]:
# Print the balanced_accuracy score of the model

print(f" {'*'*100} \n                          The accuracy Score is : {balanced_accuracy_score(y_test, y_pred_logisticR)*100:.2f} % \n {'*'*100} ")

 **************************************************************************************************** 
                          The accuracy Score is : 95.20 % 
 **************************************************************************************************** 


In [97]:
# Generate a confusion matrix for the model
confusion_matrix_1 = confusion_matrix(y_test, y_pred_logisticR)
confusion_matrix_1

array([[18663,   102],
       [   56,   563]])

In [98]:
# Print the classification report for the model
model_report_original_data = classification_report(y_test,
                                                   y_pred_logisticR, 
                                                   target_names = ["Healthy Loan","High Risk Loan"])
print(model_report_original_data)

                precision    recall  f1-score   support

  Healthy Loan       1.00      0.99      1.00     18765
High Risk Loan       0.85      0.91      0.88       619

      accuracy                           0.99     19384
     macro avg       0.92      0.95      0.94     19384
  weighted avg       0.99      0.99      0.99     19384



### Step 4: Answer the following question.

**Question:** How well does the logistic regression model predict both the `0` (healthy loan) and `1` (high-risk loan) labels?

**Answer:** The Accuracy of the model indicates that is of 99%, and that the precision for Healthy Loan is 100% but High Risk Loan is 85%, this may be affected by the data/instances support inbalance of 18765 / 619 . But Still a very accurate Model for both.

---

## Predict a Logistic Regression Model with Resampled Training Data

### Step 1: Use the `RandomOverSampler` module from the imbalanced-learn library to resample the data. Be sure to confirm that the labels have an equal number of data points. 

In [99]:
# Instantiate the random oversampler model
# # Assign a random_state parameter of 1 to the model
resampled_train_data = RandomOverSampler(random_state=1)
# Fit the original training data to the random_oversampler model
X_resample, y_resample = resampled_train_data.fit_resample(X_train,y_train)

In [100]:
# Count the distinct values of the resampled labels data
print(f"X = \n {X_resample.value_counts()} \n {'_'*100} \n {'_'*100} \n \ny=\n {y_resample.value_counts()}")

X = 
 loan_size  interest_rate  borrower_income  debt_to_income  num_of_accounts  derogatory_marks  total_debt
18500.0    10.976         83900            0.642431        12               2                 53900         209
17500.0    10.552         79900            0.624531        11               2                 49900         181
17400.0    10.510         79500            0.622642        11               2                 49500         157
18400.0    10.939         83500            0.640719        12               2                 53500         153
19300.0    11.329         87200            0.655963        12               2                 57200         140
                                                                                                           ... 
12900.0    8.594          61500            0.512195        7                1                 31500           1
           8.598          61500            0.512195        7                1                 31500      

### Step 2: Use the `LogisticRegression` classifier and the resampled data to fit the model and make predictions.

In [101]:
# Instantiate the Logistic Regression model
# Assign a random_state parameter of 1 to the model
model_logiReg_resample = LogisticRegression(random_state=1)
# Fit the model using the resampled training data
model_logiReg_resample.fit(X_resample,y_resample)
# Make a prediction using the testing data
y_pred_logiReg_resample = model_logiReg_resample.predict(X_test)

### Step 3: Evaluate the model’s performance by doing the following:

* Calculate the accuracy score of the model.

* Generate a confusion matrix.

* Print the classification report.

In [102]:
# Print the balanced_accuracy score of the model

print(f" {'*'*100} \n                          The accuracy Score is : {balanced_accuracy_score(y_test, y_pred_logiReg_resample)*100:.2f} % \n {'*'*100} ")

 **************************************************************************************************** 
                          The accuracy Score is : 99.37 % 
 **************************************************************************************************** 


In [103]:
# Generate a confusion matrix for the model
confusion_matrix_2 = confusion_matrix(y_test, y_pred_logiReg_resample)
confusion_matrix_2

array([[18649,   116],
       [    4,   615]])

In [104]:
# Print the classification report for the model
model_report_resampled_data = classification_report(y_test,
                                                   y_pred_logiReg_resample, 
                                                   target_names = ["Healthy Loan","High Risk Loan"])
print(model_report_resampled_data)

                precision    recall  f1-score   support

  Healthy Loan       1.00      0.99      1.00     18765
High Risk Loan       0.84      0.99      0.91       619

      accuracy                           0.99     19384
     macro avg       0.92      0.99      0.95     19384
  weighted avg       0.99      0.99      0.99     19384



### Step 4: Answer the following question

In [110]:
print(f"Logistic Regression Model with the Original Data \n {'-'*100}")
print(model_report_original_data)

Logistic Regression Model with the Original Data 
 ----------------------------------------------------------------------------------------------------
                precision    recall  f1-score   support

  Healthy Loan       1.00      0.99      1.00     18765
High Risk Loan       0.85      0.91      0.88       619

      accuracy                           0.99     19384
     macro avg       0.92      0.95      0.94     19384
  weighted avg       0.99      0.99      0.99     19384



In [111]:
print(f"Logistic Regression Model with Resampled Training Data \n {'-'*100}")
print(model_report_resampled_data)

Logistic Regression Model with Resampled Training Data 
 ----------------------------------------------------------------------------------------------------
                precision    recall  f1-score   support

  Healthy Loan       1.00      0.99      1.00     18765
High Risk Loan       0.84      0.99      0.91       619

      accuracy                           0.99     19384
     macro avg       0.92      0.99      0.95     19384
  weighted avg       0.99      0.99      0.99     19384



In [107]:
confusion_matrix_1

array([[18663,   102],
       [   56,   563]])

In [108]:
confusion_matrix_2

array([[18649,   116],
       [    4,   615]])

**Question:** How well does the logistic regression model, fit with oversampled data, predict both the `0` (healthy loan) and `1` (high-risk loan) labels?

**Answer:**  Both are very good models, but if we analyze the trade off, we see that the random oversampling it's in charge of balancing our data analysis,so our "High Risk Loan" decreased by 1% in precision and increasing 8% the recalls, which for this case being classification of a Loan, we are correctly identifying  High risk Loan but the new Model is more likely to falsely to identify a Loan as High Risk. We can see this by comparing the confusion matrices where the FP are greater for Resampling the data.

With this I would say that is better keep Logic regression model with the Original data.